In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import os

# === Configuration ===
data_folder = "./data"  # Folder where R0.xlsx and R1.xlsx are stored
r0_file = os.path.join(data_folder, "R0.xlsx")
r1_file = os.path.join(data_folder, "R1.xlsx")

# === Load Data ===
r0_df = pd.read_excel(r0_file, dtype=str).fillna("")
r1_df = pd.read_excel(r1_file, dtype=str).fillna("")

# Ensure 'Tag' exists
if 'Tag' not in r0_df.columns or 'Tag' not in r1_df.columns:
    raise ValueError("Both files must contain a 'Tag' column.")

# Drop duplicates and set index
r0_df = r0_df.drop_duplicates(subset='Tag').set_index('Tag')
r1_df = r1_df.drop_duplicates(subset='Tag').set_index('Tag')

# Keep R0 column order
r0_columns = list(r0_df.columns)
all_columns = sorted(set(r0_df.columns).union(set(r1_df.columns)), key=lambda x: (r0_columns.index(x) if x in r0_columns else float('inf')))

# Unique Tags
all_tags = sorted(set(r0_df.index).union(set(r1_df.index)))

# Compare Data
comparison_rows = []

for tag in all_tags:
    if tag not in r0_df.index:
        row = {"Tag": tag, "Change_Type": "✅ Added in R1"}
        row.update({col: r1_df.loc[tag].get(col, "") for col in all_columns})
        row["Change_Summary"] = ""
        comparison_rows.append(row)

    elif tag not in r1_df.index:
        row = {"Tag": tag, "Change_Type": "❌ Removed in R1"}
        row.update({col: r0_df.loc[tag].get(col, "") for col in all_columns})
        row["Change_Summary"] = ""
        comparison_rows.append(row)

    else:
        row_r0 = r0_df.loc[tag]
        row_r1 = r1_df.loc[tag]
        row_data = {"Tag": tag}
        summary = []
        changes_exist = False

        for col in all_columns:
            val_r0 = row_r0.get(col, "")
            val_r1 = row_r1.get(col, "")
            if str(val_r0).strip() != str(val_r1).strip():
                row_data[col] = f"{val_r0} → {val_r1}"
                summary.append(f"{col}: {val_r0} → {val_r1}")
                changes_exist = True
            else:
                row_data[col] = val_r1

        row_data["Change_Type"] = "✏️ Modified" if changes_exist else "No Change"
        row_data["Change_Summary"] = " | ".join(summary) if summary else ""
        comparison_rows.append(row_data)

# === Build Final DataFrame ===
comparison_df = pd.DataFrame(comparison_rows)
final_columns = ["Tag", "Change_Type"] + all_columns + ["Change_Summary"]
comparison_df = comparison_df[final_columns]

# === Row Validation Summary ===
r0_tags = set(r0_df.index)
r1_tags = set(r1_df.index)
only_in_r0 = r0_tags - r1_tags
only_in_r1 = r1_tags - r0_tags
in_both = r0_tags & r1_tags
processed_tags = set(row["Tag"] for row in comparison_rows)

print("\n=== Row Validation Summary ===")
print(f"🗂 Total Tags in R0.xlsx: {len(r0_tags)}")
print(f"🗂 Total Tags in R1.xlsx: {len(r1_tags)}")
print(f"✅ Tags Compared (common): {len(in_both)}")
print(f"➕ Added Tags (in R1 only): {len(only_in_r1)}")
print(f"➖ Removed Tags (in R0 only): {len(only_in_r0)}")
print(f"🔍 Tags Processed in Output: {len(processed_tags)}")

if processed_tags != r0_tags.union(r1_tags):
    print("⚠️ Warning: Some rows might be missing in the comparison.")
else:
    print("✅ All rows successfully compared and included.")

# === Save to Excel ===
wb = Workbook()
ws = wb.active
ws.title = "Vimal Comparison Summary"
highlight = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

for r_idx, row in enumerate(dataframe_to_rows(comparison_df, index=False, header=True), 1):
    for c_idx, value in enumerate(row, 1):
        cell = ws.cell(row=r_idx, column=c_idx, value=value)
        if isinstance(value, str) and "→" in value:
            cell.fill = highlight

timestamp = datetime.now().strftime("%d_%m_%Y_%H_%M")
output_file = os.path.join(data_folder, f"Vimal_Comparison_Summary_R0_vs_R1_{timestamp}.xlsx")
wb.save(output_file)

print(f"\n📁 Output saved as: {output_file}")



=== Row Validation Summary ===
🗂 Total Tags in R0.xlsx: 19
🗂 Total Tags in R1.xlsx: 19
✅ Tags Compared (common): 19
➕ Added Tags (in R1 only): 0
➖ Removed Tags (in R0 only): 0
🔍 Tags Processed in Output: 19
✅ All rows successfully compared and included.

📁 Output saved as: ./data\Vimal_Comparison_Summary_R0_vs_R1_05_05_2025_11_46.xlsx
